OpenVINO Local Pipelines with HuggingFace

https://python.langchain.com/docs/integrations/llms/openvino

In [ ]:
%pip install --upgrade-strategy eager "optimum[openvino,nncf]" --quiet



In [3]:
## login to huggingfacehub to get access to pretrained model 

from huggingface_hub import notebook_login, whoami

try:
    whoami()
    print('Authorization token already provided')
except OSError:
    notebook_login()

Authorization token already provided


In [1]:
from langchain_community.llms.huggingface_pipeline import HuggingFacePipeline

ov_config = {"PERFORMANCE_HINT": "LATENCY", "NUM_STREAMS": "1", "CACHE_DIR": ""}

ov_llm = HuggingFacePipeline.from_model_id(
    model_id="mistralai/Mistral-7B-Instruct-v0.2",
    task="text-generation",
    backend="openvino",
    model_kwargs={"device": "GPU", "ov_config": ov_config},
    pipeline_kwargs={"max_new_tokens": 1000},
)

INFO:nncf:NNCF initialized successfully. Supported frameworks detected: torch, onnx, openvino


Framework not specified. Using pt to export the model.


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Using the export variant default. Available variants are:
    - default: The default ONNX variant.
The model weights will be quantized to int8.
Using framework PyTorch: 2.2.2+cpu
Overriding 1 configuration item(s)
	- use_cache -> True
c:\workspace\llm\.venv\lib\site-packages\transformers\modeling_utils.py:4193: FutureWarning: `_is_quantized_training_enabled` is going to be deprecated in transformers 4.39.0. Please use `model.hf_quantizer.is_trainable` instead
  warnings.warn(
c:\workspace\llm\.venv\lib\site-packages\transformers\modeling_attn_mask_utils.py:114: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  if (input_shape[-1] > 1 or self.sliding_window is not None) and self.is_causal:
c:\workspace\llm\.venv\lib\site-packages\optimum\exporters\onnx\model_patcher.py:301:

INFO:nncf:Statistics of the bitwidth distribution:
+----------------+-----------------------------+----------------------------------------+
|   Num bits (N) | % all parameters (layers)   | % ratio-defining parameters (layers)   |
+================+=============================+========================================+
|              8 | 100% (226 / 226)            | 100% (226 / 226)                       |
+----------------+-----------------------------+----------------------------------------+


Output()

Compiling the model to GPU ...
Exception ignored in: <finalize object at 0x17a7df2ba00; dead>
Traceback (most recent call last):
  File "C:\Python39\lib\weakref.py", line 591, in __call__
    return info.func(*info.args, **(info.kwargs or {}))
  File "C:\Python39\lib\tempfile.py", line 820, in _cleanup
    cls._rmtree(name)
  File "C:\Python39\lib\tempfile.py", line 816, in _rmtree
    _shutil.rmtree(name, onerror=onerror)
  File "C:\Python39\lib\shutil.py", line 759, in rmtree
    return _rmtree_unsafe(path, onerror)
  File "C:\Python39\lib\shutil.py", line 629, in _rmtree_unsafe
    onerror(os.unlink, fullname, sys.exc_info())
  File "C:\Python39\lib\tempfile.py", line 808, in onerror
    cls._rmtree(path)
  File "C:\Python39\lib\tempfile.py", line 816, in _rmtree
    _shutil.rmtree(name, onerror=onerror)
  File "C:\Python39\lib\shutil.py", line 759, in rmtree
    return _rmtree_unsafe(path, onerror)
  File "C:\Python39\lib\shutil.py", line 610, in _rmtree_unsafe
    onerror(os.scand

In [2]:
from langchain.prompts import PromptTemplate

template = """Question: {question}

Answer: Let's think step by step."""
prompt = PromptTemplate.from_template(template)

chain = prompt | ov_llm

question = "i can fly and i can swim, what am i?"

from pprint import pprint

pprint(chain.invoke({"question": question}))

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


('Question: i can fly and i can swim, what am i?\n'
 '\n'
 "Answer: Let's think step by step. A creature that can fly is typically an "
 'animal with wings, such as a bird or a bat. A creature that can swim is '
 'typically an animal that lives in water, such as a fish or a frog. However, '
 'birds and bats cannot swim, and fish and frogs cannot fly. Therefore, the '
 'answer is not a bird, bat, fish, or frog.\n'
 '\n'
 'The only creature that can both fly and swim is a flying fish. Flying fish '
 'are not typical fish, but rather a type of fish that can jump out of the '
 'water and glide through the air for short distances using their large, '
 'wing-like pectoral fins.\n'
 '\n'
 'Therefore, the answer is a flying fish.')
